# Exercise 2: Quantum Annealing for Sampling Ionic Distributions in Disordered Materials

### Disordered Materials in Energy Research

In energy research, particularly within the domain of battery technologies, the investigation of disordered materials, such as doped materials and high-entropy oxides, has become increasingly significant. A hallmark of this disorder is the presence of vacancies alongside ions. The distribution between ions and vacancies is far from trivial; they fundamentally influence the material's behavior. This distribution is especially crucial in computational studies, where the accuracy of predictions hinges upon an accurate representation of ion-vacancy configurations. Given the extensive possible configurations, ensuring model precision is challenging; inaccuracies can substantially alter predictive results.

Thermodynamically, the configurations at the lowest energy states are of utmost significance. Materials are inherently inclined to adopt these configurations, making them critical for both understanding and predicting the behavior of the material under various conditions. The challenge, however, lies in effectively identifying these configurations from a vast solution space. Quantum Annealing presents an advanced approach to efficiently sample ionic distributions in disordered materials.

This tutorial delves into the specifics of Quantum Annealing, elucidating its capabilities and techniques for identifying low-energy states in disordered materials.

### Simplified Model: Classical Electrostatic Coulomb Energy

A foundational approach to modeling interactions in ionic systems is the consideration of pairwise Coulomb interactions. The energy $E_{\mathrm{coul}}$ is determined by the summation of interactions between every distinct pair of ions in the system. In the equation:

$$E_{coul} = \sum_{i<j} \frac{q_i\,q_j}{|r_i-r_j|}$$

$q_i$ and $q_j$ represent the charges of the ions, while $|\mathbf{r}_i-\mathbf{r}_j|$ is the distance between the ions $i$ and $j$. This equation encapsulates the essence of classical electrostatic interactions: charged entities attract or repel each other based on the inverse of the square of their separation distance. While this is a simplification and does not account for quantum effects or other complexities, it serves as a foundational starting point for understanding the energetics of ionic systems in disordered materials.

### Problem 1: Simple 2D Lattice

For this investigation, we focus on a model representing a two-dimensional lattice subjected to classical electrostatic Coulomb interactions. Theoretically, this lattice portrays a hypothetical material structure wherein we aim to distribute two lithium (Li) ions and two vacancies over a set of four designated lithium sites.

The visual representation below illustrates the lattice alongside specific ionic positions, providing clarity on the distribution of different ions within the structure. For a more detailed insight into the ionic distribution, refer to the accompanying table that enumerates the exact atomic positions.

<table style="width:100%;">
    <tr>
        <td style="width:50%; vertical-align:top;">
            <img src="2D_lattice.jpg" width="600"/>
        </td>
        <td style="width:50%; vertical-align:top;">
            <table border="1">
                <thead>
                    <tr>
                        <th>Atom</th>
                        <th>x-Coordinate</th>
                        <th>y-Coordinate</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Li<sup>+</sup></td>
                        <td>0</td>
                        <td>0</td>
                    </tr>
                    <tr>
                        <td>Li<sup>+</sup></td>
                        <td>2</td>
                        <td>0</td>
                    </tr>
                    <tr>
                        <td>Li<sup>+</sup></td>
                        <td>0</td>
                        <td>1</td>
                    </tr>
                    <tr>
                        <td>Li<sup>+</sup></td>
                        <td>2</td>
                        <td>1</td>
                    </tr>
                    <tr>
                        <td>O<sup>2-</sup></td>
                        <td>1</td>
                        <td>0</td>
                    </tr>
                    <tr>
                        <td>O<sup>2-</sup></td>
                        <td>2</td>
                        <td>0.5</td>
                    </tr>
                    <tr>
                        <td>O<sup>2-</sup></td>
                        <td>0</td>
                        <td>0.5</td>
                    </tr>
                    <tr>
                        <td>O<sup>2-</sup></td>
                        <td>1</td>
                        <td>1</td>
                    </tr>
                    <tr>
                        <td>W<sup>6+</sup></td>
                        <td>1</td>
                        <td>0.5</td>
                    </tr>
                </tbody>
            </table>
        </td>
    </tr>
</table>



#### Task 1.1: Compute the Coulomb energies of all possible configurations

The process of computing the Coulomb energy for a set of ionic configurations in our 2D lattice model hinges on the Classical Electrostatic Coulomb Energy equation:
$$E_{\mathrm{coul}}\ =\ \sum_{i<j} \frac{q_i\,q_j}{|\mathbf{r}_i-\mathbf{r}_j|}$$
The formula calculates the pairwise Coulomb interaction energy between ions in the system. The energy between two ions is proportional to the product of their charges and inversely proportional to the distance between them. Here's how we can use this concept to evaluate the energy for our lattice:

1. **Pairwise Computation**: For every distinct pair of ions, we will compute their interaction energy.
2. **Accumulation**: Sum all these pairwise energies to get the total Coulombic energy for a given ionic configuration.
3. **Iterate for All Configurations**: By varying the arrangement of Li ions and vacancies, we can compute the energy for each unique configuration.

In [35]:
import numpy as np

def energy_coul_pair(site1, site2):
    """Compute Coulomb energy between two sites."""
    q1 = site1[0]  # Charge of the first ion
    r1 = np.array(site1[1])  # Position of the first ion
    q2 = site2[0]  # Charge of the second ion
    r2 = np.array(site2[1])  # Position of the second ion
    
    # Compute pairwise interaction energy
    return q1 * q2 / np.linalg.norm(r1 - r2)

# Define the ionic sites and their charges/positions.
# Each site is represented as a tuple: (charge, [x,y])
# total_li_sites = [(1, [0, 0]), (1, [2, 0]), (1, [0, 1]), (1, [2, 1])]

#select two occupied Li sites and find all possible configurations
li_sites = [(1, [0, 0]), (1, [0, 1])] 
#li_sites = [(1, [0, 0]), (1, [2, 0])]
#li_sites = [(1, [0, 0]), (1, [2, 1])]
o_sites = [(-2, [1, 0]), (-2, [2, 0.5]), (-2, [0, 0.5]), (-2, [1, 1])]
w_site = [(6, [1, 0.5])]

# Combine all sites
all_sites = li_sites + o_sites + w_site
n_all_sites = len(all_sites)

# Initialize total energy to zero
energy_coul_total = 0
for i in range(n_all_sites):
    for j in range(i+1, n_all_sites):  # Ensures we don't double-count pairs
        energy_coul_total += energy_coul_pair(all_sites[i], all_sites[j])

print('Total Coulomb energy for the configuration: ' + str(energy_coul_total))


Total Coulomb energy for the configuration: -56.72475077703921


### Mapping Configurational Coulomb Energy onto QUBO Cost Function

Mapping the Coulomb energy into a QUBO form is crucial for processing on quantum annealers, as these devices naturally understand and solve problems represented in QUBO format. At the core, a QUBO problem seeks to minimize a function of binary variables (variables that take on values 0 or 1). The process of translating our energy problem to QUBO involves expressing the energy in terms of these binary variables. 

Let's dissect the QUBO representation of $E_{coul}$:

$$E_{coul} = c + \sum_{i} \alpha_i x_i +\sum_{i<j} \beta_{ij} x_i x_j$$

Where $x_i$ are the binary variables. In our case, these can represent the presence (1) or absence (0) of a Li ion in the ith position on the lattice.

Here's a deeper look at the three components:

1. **Constant Term, $c$**: This is the Coulomb energy arising solely from the fixed ions (like $O^{2-}$ and $W^{6+}$). Since these positions and charges are fixed, this energy doesn't change with different configurations and can be computed just once. 

2. **Linear Term, $\sum_{i} \alpha_i x_i$**: Represents the interaction energy between a single variable site (like a Li ion) and all the fixed ions in the lattice. $\alpha_i$ thus captures the cumulative interaction of the *i*-th variable site with all fixed ions.

3. **Quadratic Term, $\sum_{i<j} \beta_{ij} x_i x_j$**: This term captures the pairwise interaction energies between all possible pairs of variable sites. If two sites are occupied (i.e., both $x_i$ and $x_j$ are 1), then their interaction contributes to the total energy. $\beta_{ij}$ is the interaction energy between the *i*-th and *j*-th variable sites.

This decomposition makes the energy expression amenable to quantum annealing, by breaking it down into terms that rely on binary decision variables.

To successfully map the Coulomb energy to QUBO:
- **Compute $\alpha_i$**: For each possible Li site, compute its interaction energy with all fixed ions. This gives the $\alpha_i$ values.
  
- **Compute $\beta_{ij}$**: For every pair of possible Li sites, compute their interaction energy. These are the $\beta_{ij}$ values.

- **Compute $c$**: This involves finding the cumulative interaction energy between all fixed ions.

The matrices $\alpha$ and $\beta$, along with the constant $c$, provide the parameters necessary for the QUBO cost function. Once this function is formulated, quantum annealers can be used to find the configuration (i.e., the values of $x_i$) that minimizes this function, which in turn gives the configuration with the lowest Coulomb energy.

Please write a code for computing matrices $\alpha$, $\beta$, and constant $c$ for the 2D lattice.

In [37]:
import numpy as np

def energy_coul_pair(site1, site2):
    """Compute Coulomb energy between two sites."""
    q1 = site1[0]  # Charge of the first ion
    r1 = np.array(site1[1])  # Position of the first ion
    q2 = site2[0]  # Charge of the second ion
    r2 = np.array(site2[1])  # Position of the second ion
    
    # Compute pairwise interaction energy
    return q1 * q2 / np.linalg.norm(r1 - r2)

# Sites definition
li_sites = [(1, [0, 0]), (1, [2, 0]), (1, [0, 1]), (1, [2, 1])]
o_sites = [(-2, [1, 0]), (-2, [2, 0.5]), (-2, [0, 0.5]), (-2, [1, 1])]
w_site = [(6, [1, 0.5])]

# Compute c: Coulomb energy from fixed atoms
def compute_c(o_sites, w_site):
    c_energy = 0
    # Only considering O-W interactions since O-O are fixed and won't affect optimization
    for o in o_sites:
        c_energy += energy_coul_pair(o, w_site[0])
    return c_energy

# Compute alpha: The Coulomb energy from 1 distributed Li atom and fixed atoms
def compute_alpha(li_sites, o_sites, w_site):
    alpha = []
    for li in li_sites:
        alpha_energy = 0
        for o in o_sites:
            alpha_energy += energy_coul_pair(li, o)
        alpha_energy += energy_coul_pair(li, w_site[0])
        alpha.append(alpha_energy)
    return np.array(alpha)

# Compute beta: The Coulomb energy from distributed atoms (Li^+ - Li^+)
def compute_beta(li_sites):
    beta = np.zeros((len(li_sites), len(li_sites)))
    for i in range(len(li_sites)):
        for j in range(i+1, len(li_sites)):
            beta[i, j] = energy_coul_pair(li_sites[i], li_sites[j])
    return beta

# Compute the QUBO parameters
c = compute_c(o_sites, w_site)
alpha = compute_alpha(li_sites, o_sites, w_site)
beta = compute_beta(li_sites)

# Print the QUBO parameters
print("c =", c)
print("alpha =", alpha)
print("beta matrix =")
print(beta)

c = -72.0
alpha = [-3.01779292 -3.01779292 -3.01779292 -3.01779292]
beta matrix =
[[0.        0.5       1.        0.4472136]
 [0.        0.        0.4472136 1.       ]
 [0.        0.        0.        0.5      ]
 [0.        0.        0.        0.       ]]


In [38]:
# Calculate the QUBO cost for a given configuration of x
def E_QUBO(x, alpha, beta, c):
    E = c
    for i in range(len(x)):
        E += alpha[i] * x[i]
        for j in range(i+1, len(x)):
            E += beta[i, j] * x[i] * x[j]
    return E

# Function to get all possible configurations of Li distribution
def get_configurations(n_sites, n_li):
    from itertools import combinations
    configs = list(combinations(range(n_sites), n_li))
    binary_configs = []
    for config in configs:
        binary = [0] * n_sites
        for i in config:
            binary[i] = 1
        binary_configs.append(binary)
    return binary_configs


configurations = get_configurations(len(li_sites), 2)  # 2 Li atoms out of 4 possible sites
print("Generated configurations:", configurations)
for config in configurations:
    E = E_QUBO(config, alpha, beta, c)
    print(f"E_QUBO for x = {config} is {E}")

Generated configurations: [[1, 1, 0, 0], [1, 0, 1, 0], [1, 0, 0, 1], [0, 1, 1, 0], [0, 1, 0, 1], [0, 0, 1, 1]]
E_QUBO for x = [1, 1, 0, 0] is -77.53558583303786
E_QUBO for x = [1, 0, 1, 0] is -77.03558583303786
E_QUBO for x = [1, 0, 0, 1] is -77.5883722375379
E_QUBO for x = [0, 1, 1, 0] is -77.5883722375379
E_QUBO for x = [0, 1, 0, 1] is -77.03558583303786
E_QUBO for x = [0, 0, 1, 1] is -77.53558583303786
